## Read .ghg metadata

In [9]:
import preProcessing
import importlib
import time
importlib.reload(preProcessing)
T1 = time.time()

pr = preProcessing.read_ALL('BB',2022,processes=1,Test=12,reset=1)
T2 = time.time()
print('\n')
print(T2-T1)


col_co2 [26]
col_h2o [28]
col_ch4 [50]
col_n2o []
col_int_t_1 [31]
col_int_t_2 [32]
col_int_p [15]
col_air_t [51]
col_air_p [52]
col_cell_t [30]
col_diag_75 [4]
col_diag_72 []
col_diag_77 [54]
col_diag_anem []
C:/highfreq/BB/metadata/File_Inventory.csv


6.602632522583008


In [24]:
import setupEP as eP
import importlib
import time
importlib.reload(eP)

# T1 = time.time()
mR = eP.makeRun('ep_Templates/DefaultSettings.eddypro','BB')
mR.runDates(['2022-11-06','2022-11-08'],'test')


DEFAULT
Project
RawProcess_General
DEFAULT
Project
RawProcess_General
DEFAULT
Project
RawProcess_General


In [15]:
mR.ini['Paths']['raw']

'C:/highfreq/SITE/raw/YEAR/'

In [13]:
# # from HelperFunctions import progressbar

# # pb = progressbar(15, "Computing: ", 40)


# mR.inventory.index.date
# # import time    
# # # pb = progressbar()
# # for i in range(15):
# #     time.sleep(0.1) # any code you need
# #     pb.step()

mR.Year

2022